In [1]:
from google.colab import drive
drive.mount('/content/drive')
orgpath = '/content/drive/MyDrive/sayandeep/'

Mounted at /content/drive


In [2]:
import os
import numpy as np
import cv2
from tensorflow.keras.utils import Sequence
from sklearn.metrics import confusion_matrix, jaccard_score
from tensorflow.keras.callbacks import ModelCheckpoint, LearningRateScheduler, ReduceLROnPlateau
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Conv2D, BatchNormalization, Activation, MaxPooling2D, UpSampling2D, Concatenate
from tensorflow.keras.layers import LayerNormalization, MultiHeadAttention
import tensorflow.keras.backend as K
import tensorflow as tf

In [3]:
FOLDER_PATH = orgpath + 'Processed_Dataset/'
image_size = 256
batch_size = 64
epochs = 50


train_path = FOLDER_PATH + "training/"
validation_path = FOLDER_PATH + "validation/"
test_path = FOLDER_PATH + "testing/"

#NEW CODE BELOW

Underlying Code is a data generator tailored for tasks where you need segmentation (images and masks).  

For modification, change things like:
image_path = os.path.join(self.path, "images", "img", id_name)

this means image path = path/images/img/1.png,2.pngetc
if you change directory structure take note.

In [4]:
#tridibeshoblikheche
class DataGen(Sequence):
    def __init__(self, ids, path, batch_size=2, image_size=256):
        self.ids = ids
        self.path = path
        self.batch_size = batch_size
        self.image_size = image_size
        self.on_epoch_end()

    def __load__(self, id_name):
        image_path = os.path.join(self.path, "images", "img", id_name)
        mask_path = os.path.join(self.path, "mask", "img", id_name)
        image = cv2.imread(image_path)
        if image is None or image.size == 0:
            return None, None
        image = cv2.resize(image, (self.image_size, self.image_size)) / 255.0

        mask = cv2.imread(mask_path, cv2.IMREAD_GRAYSCALE)
        if mask is None or mask.size == 0:
            return None, None
        mask = cv2.resize(mask, (self.image_size, self.image_size)) / 255.0
        mask = np.expand_dims(mask, axis=-1)

        return image, mask

    def __getitem__(self, index):
        files_batch = self.ids[index*self.batch_size : (index+1)*self.batch_size]
        images, masks = [], []
        for id_name in files_batch:
            img, msk = self.__load__(id_name)
            if img is not None and msk is not None:
                images.append(img)
                masks.append(msk)
        return np.array(images), np.array(masks)

    def on_epoch_end(self):
        np.random.shuffle(self.ids)

    def __len__(self):
        return int(np.ceil(len(self.ids) / float(self.batch_size)))

In [ ]:
# # Define the paths
# dataset_path = '/content/drive/My Drive/dataset'
# image_dirs = ['(0c) Cropped10', '(0i) FolderXGray']
# mask_dirs = ['(5a) LumenFarWall', '(5b) MA-FarWall']

# # Load images and masks
# def load_images_and_masks(image_dirs, mask_dirs, dataset_path, target_size=(256, 256)):
#     images = []
#     masks = []

#     for image_dir, mask_dir in zip(image_dirs, mask_dirs):
#         image_path = os.path.join(dataset_path, image_dir)
#         mask_path = os.path.join(dataset_path, mask_dir)

#         image_files = sorted(os.listdir(image_path))
#         mask_files = sorted(os.listdir(mask_path))

#         for image_file, mask_file in zip(image_files, mask_files):
#             image = cv2.imread(os.path.join(image_path, image_file), cv2.IMREAD_GRAYSCALE)
#             mask = np.loadtxt(os.path.join(mask_path, mask_file))

#             # Resize image and mask
#             image = cv2.resize(image, target_size)
#             mask = cv2.resize(mask, target_size)

#             # Expand dimensions to match model input
#             image = np.expand_dims(image, axis=-1)
#             mask = np.expand_dims(mask, axis=-1)

#             images.append(image)
#             masks.append(mask)

#     return np.array(images), np.array(masks)

# # Normalize images and masks
# def normalize_images_and_masks(images, masks):
#     images = images / 255.0
#     masks = masks / np.max(masks)
#     return images, masks

# # Load and preprocess the data
# images, masks = load_images_and_masks(image_dirs, mask_dirs, dataset_path)
# images, masks = normalize_images_and_masks(images, masks)

# # Check the shape of images and masks
# print(f'Images shape: {images.shape}')
# print(f'Masks shape: {masks.shape}')


In [ ]:
# # Split the data into training and validation sets
# X_train, X_val, y_train, y_val = train_test_split(images, masks, test_size=0.2, random_state=42)

# # Check the shape of the training and validation sets
# print(f'X_train shape: {X_train.shape}')
# print(f'X_val shape: {X_val.shape}')
# print(f'y_train shape: {y_train.shape}')
# print(f'y_val shape: {y_val.shape}')


In [ ]:
# # Data Augmentation
# data_gen_args = dict(rotation_range=10,
#                      width_shift_range=0.1,
#                      height_shift_range=0.1,
#                      shear_range=0.1,
#                      zoom_range=0.1,
#                      horizontal_flip=True,
#                      fill_mode='nearest')

# image_datagen = ImageDataGenerator(**data_gen_args)
# mask_datagen = ImageDataGenerator(**data_gen_args)

# image_datagen.fit(X_train, augment=True, seed=42)
# mask_datagen.fit(y_train, augment=True, seed=42)

# image_generator = image_datagen.flow(X_train, batch_size=8, seed=42)
# mask_generator = mask_datagen.flow(y_train, batch_size=8, seed=42)

# train_generator = zip(image_generator, mask_generator)


#NEW CODE BELOW
providing some helper functions, you may just run them. for comparing  models previously made by amit sir, you will need. by me, you will not but good to have.

apart from scheduler, dice, and visulaise output, load test, it is redundant (basically, metric functions are redundant but KEEP, custom metrics, cannot load some models for compariosn without them)

In [5]:
def iou(y_true, y_pred, smooth=1):
    intersection = K.sum(K.abs(y_true * y_pred), axis=[1,2,3])
    union = K.sum(y_true,[1,2,3])+K.sum(y_pred,[1,2,3])-intersection
    iou = K.mean((intersection + smooth) / (union + smooth), axis=0)
    return iou

def F1(y_true, y_pred, smooth=1):
    intersection = K.sum(y_true * y_pred, axis=[1, 2, 3])
    union = K.sum(y_true, axis=[1, 2, 3]) + K.sum(y_pred, axis=[1, 2, 3])
    dice = K.mean((2. * intersection + smooth) / (union + smooth), axis=0)
    return dice

def dice_coef(y_true, y_pred, smooth = 0.00001):
    y_true_f = K.flatten(y_true)
    y_pred_f = K.flatten(y_pred)
    intersection = K.sum(y_true_f * y_pred_f)
    return (2. * intersection + smooth) / (K.sum(y_true_f) + K.sum(y_pred_f) + smooth)

def recall(y_true, y_pred):
    true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
    possible_positives = K.sum(K.round(K.clip(y_true, 0, 1)))
    recall = true_positives / (possible_positives + K.epsilon())
    return recall

def precision(y_true, y_pred):
    true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
    predicted_positives = K.sum(K.round(K.clip(y_pred, 0, 1)))
    precision = true_positives / (predicted_positives + K.epsilon())
    return precision

def dice_coef_loss(y_true, y_pred):
    return -dice_coef(y_true, y_pred)

def dice_loss(y_true, y_pred):
    y_true_f = tf.keras.backend.flatten(y_true)
    y_pred_f = tf.keras.backend.flatten(y_pred)
    intersection = tf.keras.backend.sum(y_true_f * y_pred_f)
    return 1 - (2. * intersection + 1) / (tf.keras.backend.sum(y_true_f) + tf.keras.backend.sum(y_pred_f) + 1)

def bce_dice_loss(y_true, y_pred):
    bce = tf.keras.losses.BinaryCrossentropy()(y_true, y_pred)
    dice = dice_loss(y_true, y_pred)
    return bce + dice

#till here it is mostly redundant, below is real code but like i said, keep.
def load_test_data(batch_size=10):
    test_ids = os.listdir(os.path.join(test_path, "images", "img"))
    gen = DataGen(test_ids, test_path, batch_size=batch_size, image_size=image_size)
    print("Test Data Loaded")
    return gen, len(test_ids)

def scheduler(epoch, lr):
    if epoch < 30:
        lr = 0.001
        return lr
    if epoch < 50:
        return 0.0005
    return 0.0001

def visualize_output(x, y, y_pred, tot, idx, fig, title_flag=False):
    contours1, hierarchy = cv2.findContours(y[0].astype(np.uint8), cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_NONE)
    contours2, hierarchy = cv2.findContours(y_pred[0].astype(np.uint8), cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_NONE)
    cv2.drawContours(x[0], contours1, -1, (0, 255, 0), 3)
    cv2.drawContours(x[0], contours2, -1, (255, 0, 0), 3)
    ax = fig.add_subplot(tot, 5, idx)
    ax.imshow(np.reshape(x[0], (image_size, image_size, 3)))

def evaluate_model(gen, model):
    results = {}
    y_true_all = []
    y_pred_all = []
    num_tests = len(gen)
    for i in range(num_tests):
        x, y_true = gen[i]
        y_pred_prob = model.predict(x)
        y_pred = (y_pred_prob > 0.8).astype(int)
        y_true = (y_true > 0.5).astype(int)
        y_true_all.extend(y_true.flatten())
        y_pred_all.extend(y_pred.flatten())
    y_true_all = np.array(y_true_all)
    y_pred_all = np.array(y_pred_all)
    tn, fp, fn, tp = confusion_matrix(y_true_all, y_pred_all).ravel()
    precision = tp / (tp + fp)
    recall = tp / (tp + fn)
    specificity = tn / (tn + fp)
    iou = jaccard_score(y_true_all, y_pred_all)
    out = model.evaluate(gen, steps=num_tests)
    last_metric = out[-1]
    results = {
        'Precision': precision,
        'Recall': recall,
        'Specificity': specificity,
        'IoU': iou,
        'Eval Metric': last_metric
    }
    print(results)

In [ ]:
# inputs = tf.keras.layers.Input(shape=(256, 256, 3))

# # Encoder
# c1 = tf.keras.layers.Conv2D(64, (3, 3), activation='relu', padding='same')(inputs)
# c1 = tf.keras.layers.Conv2D(64, (3, 3), activation='relu', padding='same')(c1)
# p1 = tf.keras.layers.MaxPooling2D((2, 2))(c1)

# c2 = tf.keras.layers.Conv2D(128, (3, 3), activation='relu', padding='same')(p1)
# c2 = tf.keras.layers.Conv2D(128, (3, 3), activation='relu', padding='same')(c2)
# p2 = tf.keras.layers.MaxPooling2D((2, 2))(c2)

# c3 = tf.keras.layers.Conv2D(256, (3, 3), activation='relu', padding='same')(p2)
# c3 = tf.keras.layers.Conv2D(256, (3, 3), activation='relu', padding='same')(c3)
# p3 = tf.keras.layers.MaxPooling2D((2, 2))(c3)

# c4 = tf.keras.layers.Conv2D(512, (3, 3), activation='relu', padding='same')(p3)
# c4 = tf.keras.layers.Conv2D(512, (3, 3), activation='relu', padding='same')(c4)
# p4 = tf.keras.layers.MaxPooling2D((2, 2))(c4)

# # Bottleneck
# bn = tf.keras.layers.Conv2D(1024, (3, 3), activation='relu', padding='same')(p4)
# bn = tf.keras.layers.Conv2D(1024, (3, 3), activation='relu', padding='same')(bn)

# # Decoder
# u6 = tf.keras.layers.Conv2DTranspose(512, (2, 2), strides=(2, 2), padding='same')(bn)
# u6 = tf.keras.layers.concatenate([u6, c4])
# c6 = tf.keras.layers.Conv2D(512, (3, 3), activation='relu', padding='same')(u6)
# c6 = tf.keras.layers.Conv2D(512, (3, 3), activation='relu', padding='same')(c6)

# u7 = tf.keras.layers.Conv2DTranspose(256, (2, 2), strides=(2, 2), padding='same')(c6)
# u7 = tf.keras.layers.concatenate([u7, c3])
# c7 = tf.keras.layers.Conv2D(256, (3, 3), activation='relu', padding='same')(u7)
# c7 = tf.keras.layers.Conv2D(256, (3, 3), activation='relu', padding='same')(c7)

# u8 = tf.keras.layers.Conv2DTranspose(128, (2, 2), strides=(2, 2), padding='same')(c7)
# # u8 = tf.keras.layers.concatenate([u8, c2])
# c8 = tf.keras.layers.Conv2D(128, (3, 3), activation='relu', padding='same')(u8)
# c8 = tf.keras.layers.Conv2D(128, (3, 3), activation='relu', padding='same')(c8)

# u9 = tf.keras.layers.Conv2DTranspose(64, (2, 2), strides=(2, 2), padding='same')(c8)
# u9 = tf.keras.layers.concatenate([u9, c1])
# c9 = tf.keras.layers.Conv2D(64, (3, 3), activation='relu', padding='same')(u9)
# c9 = tf.keras.layers.Conv2D(64, (3, 3), activation='relu', padding='same')(c9)

# outputs = tf.keras.layers.Conv2D(1, (1, 1), activation='sigmoid')(c9)

# model = tf.keras.models.Model(inputs=[inputs], outputs=[outputs])
# model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=0.001),
#               loss='binary_crossentropy',
#               metrics=['accuracy'])


#NEW CODE BELOW

TRAING CODE IS NOT THE BEST. USE THE ONE I GIVE BELOW. DEPRECATED WARNING CAN COME, IN FUTURE MODIFY.


In [ ]:
# # Load training and validation data
# train_ids = os.listdir(os.path.join(train_path, "images", "img",))  #keeep track. do not fuck up directory structure
# val_ids = os.listdir(os.path.join(validation_path, "images", "img",))

# train_gen = DataGen(train_ids, train_path, batch_size=batch_size, image_size=image_size)
# val_gen = DataGen(val_ids, validation_path, batch_size=batch_size, image_size=image_size)

# # Callbacks
# checkpoint = ModelCheckpoint('unet_mhsa_best_model.h5', save_best_only=True, monitor='val_loss', mode='min')  #Change model name
# lr_scheduler = LearningRateScheduler(scheduler)
# reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=5, min_lr=0.00001)
# # Train the model
# history = model.fit(
#     train_gen,
#     validation_data=val_gen,
#     epochs=epochs,
#     callbacks=[checkpoint, lr_scheduler, reduce_lr]
# )

# model.save(orgpath + 'unet_mhsa_final_model') #change model name
# print("Model saved successfully!")

KeyboardInterrupt: 

In [ ]:
# import os
# import numpy as np
# import matplotlib.pyplot as plt
# import cv2
# import tensorflow as tf
# from keras import backend as K
# from sklearn.metrics import confusion_matrix, jaccard_score

# # Function to mount Google Drive
# from google.colab import drive
# drive.mount('/content/drive')

# # Path to the model
# model_path = '/content/drive/MyDrive/sayandeep/sayandeep_unet'

# # Load the model
# model = tf.keras.models.load_model(model_path, compile=False)

# # Define custom metrics and loss functions if necessary
# def iou(y_true, y_pred, smooth=1):
#     intersection = K.sum(K.abs(y_true * y_pred), axis=[1,2,3])
#     union = K.sum(y_true,[1,2,3]) + K.sum(y_pred,[1,2,3]) - intersection
#     iou = K.mean((intersection + smooth) / (union + smooth), axis=0)
#     return iou

# def F1(y_true, y_pred, smooth=1):
#     intersection = K.sum(y_true * y_pred, axis=[1, 2, 3])
#     union = K.sum(y_true, axis=[1, 2, 3]) + K.sum(y_pred, axis=[1, 2, 3])
#     dice = K.mean((2. * intersection + smooth) / (union + smooth), axis=0)
#     return dice

# def dice_loss(y_true, y_pred):
#     y_true_f = tf.keras.backend.flatten(y_true)
#     y_pred_f = tf.keras.backend.flatten(y_pred)
#     intersection = tf.keras.backend.sum(y_true_f * y_pred_f)
#     return 1 - (2. * intersection + 1) / (tf.keras.backend.sum(y_true_f) + tf.keras.backend.sum(y_pred_f) + 1)

# def bce_dice_loss(y_true, y_pred):
#     bce = tf.keras.losses.BinaryCrossentropy()(y_true, y_pred)
#     dice = dice_loss(y_true, y_pred)
#     return bce + dice

# # Recompile the model if necessary
# model.compile(optimizer='adam', loss=bce_dice_loss, metrics=[iou, F1, 'accuracy'])

# # Function to load test data (modify as per your data generator)
# def load_test_data(batch_size=10):
#     test_ids = os.listdir(os.path.join(test_path, "images", "img"))
#     gen = DataGen(test_ids, test_path, batch_size=batch_size, image_size=image_size)
#     print("Test Data Loaded")
#     return gen, len(test_ids)

# # Function to visualize the output
# def visualize_output(x, y, y_pred, tot, idx, fig, title_flag=False):
#     contours1, _ = cv2.findContours(y[0].astype(np.uint8), cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_NONE)
#     contours2, _ = cv2.findContours(y_pred[0].astype(np.uint8), cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_NONE)
#     img_with_contours = x[0].copy()
#     cv2.drawContours(img_with_contours, contours1, -1, (0, 255, 0), 3)
#     cv2.drawContours(img_with_contours, contours2, -1, (255, 0, 0), 3)
#     ax = fig.add_subplot(tot, 3, idx*3 + 1)
#     ax.imshow(np.reshape(x[0], (image_size, image_size, 3)))
#     if title_flag:
#         ax.set_title('Original Image')
#     ax = fig.add_subplot(tot, 3, idx*3 + 2)
#     ax.imshow(np.reshape(y[0], (image_size, image_size)), cmap='gray')
#     if title_flag:
#         ax.set_title('True Mask')
#     ax = fig.add_subplot(tot, 3, idx*3 + 3)
#     ax.imshow(np.reshape(y_pred[0], (image_size, image_size)), cmap='gray')
#     if title_flag:
#         ax.set_title('Predicted Mask')

# # Evaluation and visualization function
# def evaluate_and_visualize_model(gen, model):
#     results = {}
#     y_true_all = []
#     y_pred_all = []
#     num_tests = len(gen)

#     fig = plt.figure(figsize=(15, num_tests * 5))

#     for i in range(num_tests):
#         x, y_true = gen[i]
#         y_pred_prob = model.predict(x)
#         y_pred = (y_pred_prob > 0.8).astype(int)
#         y_true = (y_true > 0.5).astype(int)

#         visualize_output(x, y_true, y_pred, num_tests, i, fig, title_flag=(i == 0))

#         y_true_all.extend(y_true.flatten())
#         y_pred_all.extend(y_pred.flatten())

#     y_true_all = np.array(y_true_all)
#     y_pred_all = np.array(y_pred_all)

#     tn, fp, fn, tp = confusion_matrix(y_true_all, y_pred_all).ravel()
#     precision = tp / (tp + fp)
#     recall = tp / (tp + fn)
#     specificity = tn / (tn + fp)
#     iou = jaccard_score(y_true_all, y_pred_all)
#     out = model.evaluate(gen, steps=num_tests)
#     last_metric = out[-1]
#     results = {
#         'Precision': precision,
#         'Recall': recall,
#         'Specificity': specificity,
#         'IoU': iou,
#         'Eval Metric': last_metric
#     }
#     print(results)
#     plt.show()

# # Load your test data generator
# test_gen, num_test_samples = load_test_data(batch_size=10)

# # Evaluate and visualize the model
# evaluate_and_visualize_model(test_gen, model)


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
Test Data Loaded


KeyboardInterrupt: 

<Figure size 1500x13500 with 0 Axes>

In [ ]:
# import tensorflow as tf
# from tensorflow.keras.layers import GlobalAveragePooling2D, Reshape, Dense, multiply, Conv2D, BatchNormalization, Activation, MaxPooling2D, Concatenate, UpSampling2D, Input
# from tensorflow.keras.models import Model

# def se_block(input_tensor, reduction_ratio=16):
#     channel_axis = -1
#     filters = input_tensor.shape[channel_axis]
#     se_shape = (1, 1, filters)

#     se = GlobalAveragePooling2D()(input_tensor)
#     se = Reshape(se_shape)(se)
#     se = Dense(filters // reduction_ratio, activation='relu', use_bias=False)(se)
#     se = Dense(filters, activation='sigmoid', use_bias=False)(se)

#     x = multiply([input_tensor, se])
#     return x


In [ ]:
# def encoder_block(input_tensor, filters, kernel_size=(3, 3), padding='same', strides=1):
#     x = Conv2D(filters, kernel_size, padding=padding, strides=strides)(input_tensor)
#     x = BatchNormalization()(x)
#     x = Activation('relu')(x)
#     x = Conv2D(filters, kernel_size, padding=padding, strides=strides)(x)
#     x = BatchNormalization()(x)
#     x = Activation('relu')(x)
#     x = se_block(x)  # Apply SE block
#     return x

# def decoder_block(input_tensor, skip_tensor, filters, kernel_size=(3, 3), padding='same', strides=1):
#     x = Conv2D(filters, kernel_size, padding=padding, strides=strides)(input_tensor)
#     x = BatchNormalization()(x)
#     x = Activation('relu')(x)
#     x = Conv2D(filters, kernel_size, padding=padding, strides=strides)(x)
#     x = BatchNormalization()(x)
#     x = Activation('relu')(x)
#     x = UpSampling2D(size=(2, 2))(x)
#     x = Concatenate()([x, skip_tensor])
#     return x

# def build_unet_with_se(input_shape, num_classes):
#     inputs = Input(input_shape)

#     # Encoder
#     e1 = encoder_block(inputs, 64)
#     p1 = MaxPooling2D(pool_size=(2, 2))(e1)

#     e2 = encoder_block(p1, 128)
#     p2 = MaxPooling2D(pool_size=(2, 2))(e2)

#     e3 = encoder_block(p2, 256)
#     p3 = MaxPooling2D(pool_size=(2, 2))(e3)

#     e4 = encoder_block(p3, 512)
#     p4 = MaxPooling2D(pool_size=(2, 2))(e4)

#     # Bottleneck
#     b = encoder_block(p4, 1024)

#     # Decoder
#     d4 = decoder_block(b, e4, 512)
#     d3 = decoder_block(d4, e3, 256)
#     d2 = decoder_block(d3, e2, 128)
#     d1 = decoder_block(d2, e1, 64)

#     outputs = Conv2D(num_classes, (1, 1), activation='sigmoid')(d1)

#     model = Model(inputs, outputs)
#     return model


In [ ]:
# input_shape = (256, 256, 3)  # Adjust this according to your input shape
# num_classes = 1  # Adjust this according to your number of classes

# # Build the model
# model = build_unet_with_se(input_shape, num_classes)

# # Compile the model
# model.compile(optimizer='adam', loss=bce_dice_loss, metrics=[iou, F1, 'accuracy'])

# # Print the model summary
# model.summary()

# # Load training and validation data
# train_path = '/content/drive/My Drive/sayandeep/Processed_Dataset/training'  # Update this with the actual path
# validation_path = '/content/drive/My Drive/sayandeep/Processed_Dataset/validation'  # Update this with the actual path
# batch_size = 16  # Adjust batch size as needed
# image_size = 256  # Adjust image size as needed

# train_ids = os.listdir(os.path.join(train_path, "images", "img"))
# val_ids = os.listdir(os.path.join(validation_path, "images", "img"))

# train_gen = DataGen(train_ids, train_path, batch_size=batch_size, image_size=image_size)
# val_gen = DataGen(val_ids, validation_path, batch_size=batch_size, image_size=image_size)

# # Callbacks
# checkpoint = ModelCheckpoint('/content/drive/My Drive/sayandeep/sayandeep_unet.h5', save_best_only=True, monitor='val_loss', mode='min')
# lr_scheduler = LearningRateScheduler(scheduler)
# reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=5, min_lr=0.00001)

# # Train the model
# history = model.fit(
#     train_gen,
#     validation_data=val_gen,
#     epochs=50,
#     callbacks=[checkpoint, lr_scheduler, reduce_lr]
# )

# # Save the model as TensorFlow SavedModel
# model_save_path_tf = '/content/drive/My Drive/sayandeep/sayandeep_unet_attention'
# model.save(model_save_path_tf, save_format='tf')
# print(f"Model saved to {model_save_path_tf}")

Model: "model_1"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 input_2 (InputLayer)        [(None, 256, 256, 3)]        0         []                            
                                                                                                  
 conv2d_19 (Conv2D)          (None, 256, 256, 64)         1792      ['input_2[0][0]']             
                                                                                                  
 batch_normalization (Batch  (None, 256, 256, 64)         256       ['conv2d_19[0][0]']           
 Normalization)                                                                                   
                                                                                                  
 activation (Activation)     (None, 256, 256, 64)         0         ['batch_normalization[0]

In [ ]:
# import matplotlib.pyplot as plt
# import numpy as np
# import tensorflow as tf
# from sklearn.metrics import confusion_matrix, jaccard_score
# from tensorflow.keras.utils import plot_model

# # Function to evaluate the model
# def evaluate_model(model, test_gen):
#     results = {}
#     y_true_all = []
#     y_pred_all = []
#     num_tests = len(test_gen)
#     for i in range(num_tests):
#         x, y_true = test_gen[i]
#         y_pred_prob = model.predict(x)
#         y_pred = (y_pred_prob > 0.8).astype(int)
#         y_true = (y_true > 0.5).astype(int)
#         y_true_all.extend(y_true.flatten())
#         y_pred_all.extend(y_pred.flatten())
#     y_true_all = np.array(y_true_all)
#     y_pred_all = np.array(y_pred_all)
#     tn, fp, fn, tp = confusion_matrix(y_true_all, y_pred_all).ravel()
#     precision = tp / (tp + fp)
#     recall = tp / (tp + fn)
#     specificity = tn / (tn + fp)
#     iou = jaccard_score(y_true_all, y_pred_all)
#     out = model.evaluate(test_gen, steps=num_tests)
#     last_metric = out[-1]
#     results = {
#         'Precision': precision,
#         'Recall': recall,
#         'Specificity': specificity,
#         'IoU': iou,
#         'Eval Metric': last_metric
#     }
#     print(results)
#     return results

# # Function to visualize model performance
# def plot_metrics(history):
#     # Plot training & validation loss values
#     plt.figure(figsize=(12, 4))
#     plt.subplot(1, 2, 1)
#     plt.plot(history.history['loss'])
#     plt.plot(history.history['val_loss'])
#     plt.title('Model loss')
#     plt.ylabel('Loss')
#     plt.xlabel('Epoch')
#     plt.legend(['Train', 'Validation'], loc='upper left')

#     # Plot training & validation accuracy values
#     plt.subplot(1, 2, 2)
#     plt.plot(history.history['accuracy'])
#     plt.plot(history.history['val_accuracy'])
#     plt.title('Model accuracy')
#     plt.ylabel('Accuracy')
#     plt.xlabel('Epoch')
#     plt.legend(['Train', 'Validation'], loc='upper left')

#     plt.show()

# # Function to visualize predictions
# def visualize_predictions(model, test_gen, num_images=5):
#     for i in range(num_images):
#         x, y_true = test_gen[i]
#         y_pred_prob = model.predict(x)
#         y_pred = (y_pred_prob > 0.8).astype(int)

#         plt.figure(figsize=(15, 5))

#         # Original Image
#         plt.subplot(1, 3, 1)
#         plt.imshow(np.reshape(x[0], (image_size, image_size, 3)))
#         plt.title('Original Image')

#         # Ground Truth
#         plt.subplot(1, 3, 2)
#         plt.imshow(np.reshape(y_true[0], (image_size, image_size)), cmap='gray')
#         plt.title('Ground Truth')

#         # Predicted Mask
#         plt.subplot(1, 3, 3)
#         plt.imshow(np.reshape(y_pred[0], (image_size, image_size)), cmap='gray')
#         plt.title('Predicted Mask')

#         plt.show()

# # Load the saved model
# model_save_path_tf = '/content/drive/My Drive/sayandeep/sayandeep_unet_attention'
# model = tf.keras.models.load_model(model_save_path_tf, custom_objects={'bce_dice_loss': bce_dice_loss, 'iou': iou, 'F1': F1})

# # Load test data
# test_path = '/content/drive/My Drive/sayandeep/Processed_Dataset/testing'
# batch_size = 16  # Adjust batch size as needed
# image_size = 256  # Adjust image size as needed

# test_ids = os.listdir(os.path.join(test_path,"images","img",))
# test_gen = DataGen(test_ids, test_path, batch_size=batch_size, image_size=image_size)

# # Evaluate the model
# results = evaluate_model(model, test_gen)

# # Plot the training history
# plot_metrics(history)

# # Visualize predictions
# visualize_predictions(model, test_gen, num_images=5)


KeyboardInterrupt: 

In [ ]:
# import tensorflow as tf
# from tensorflow.keras.layers import Conv2D, Dense, Activation, Reshape, multiply, Add, GlobalAveragePooling2D, GlobalMaxPooling2D, Concatenate, Lambda

# def cbam_block(input_tensor, reduction_ratio=16):
#     channel_axis = -1
#     filters = input_tensor.shape[channel_axis]

#     # Channel Attention
#     avg_pool = GlobalAveragePooling2D()(input_tensor)
#     avg_pool = Reshape((1, 1, filters))(avg_pool)
#     avg_pool = Dense(filters // reduction_ratio, activation='relu', kernel_initializer='he_normal', use_bias=False)(avg_pool)
#     avg_pool = Dense(filters, activation='sigmoid', kernel_initializer='he_normal', use_bias=False)(avg_pool)

#     max_pool = GlobalMaxPooling2D()(input_tensor)
#     max_pool = Reshape((1, 1, filters))(max_pool)
#     max_pool = Dense(filters // reduction_ratio, activation='relu', kernel_initializer='he_normal', use_bias=False)(max_pool)
#     max_pool = Dense(filters, activation='sigmoid', kernel_initializer='he_normal', use_bias=False)(max_pool)

#     channel_attention = Add()([avg_pool, max_pool])

#     x = multiply([input_tensor, channel_attention])

#     # Spatial Attention
#     avg_pool = Lambda(lambda x: tf.reduce_mean(x, axis=-1, keepdims=True))(x)
#     max_pool = Lambda(lambda x: tf.reduce_max(x, axis=-1, keepdims=True))(x)
#     spatial_attention = Concatenate(axis=-1)([avg_pool, max_pool])
#     spatial_attention = Conv2D(1, (7, 7), padding='same', activation='sigmoid', kernel_initializer='he_normal', use_bias=False)(spatial_attention)

#     x = multiply([x, spatial_attention])

#     return x


In [ ]:
# from tensorflow.keras.layers import Input, MaxPooling2D, Concatenate, UpSampling2D, BatchNormalization, Activation

# def encoder_block(input_tensor, filters, kernel_size=(3, 3), padding='same', strides=1):
#     x = Conv2D(filters, kernel_size, padding=padding, strides=strides)(input_tensor)
#     x = BatchNormalization()(x)
#     x = Activation('relu')(x)
#     x = Conv2D(filters, kernel_size, padding=padding, strides=strides)(x)
#     x = BatchNormalization()(x)
#     x = Activation('relu')(x)
#     x = cbam_block(x)  # Apply CBAM block
#     return x

# def decoder_block(input_tensor, skip_tensor, filters, kernel_size=(3, 3), padding='same', strides=1):
#     x = Conv2D(filters, kernel_size, padding=padding, strides=strides)(input_tensor)
#     x = BatchNormalization()(x)
#     x = Activation('relu')(x)
#     x = Conv2D(filters, kernel_size, padding=padding, strides=strides)(x)
#     x = BatchNormalization()(x)
#     x = Activation('relu')(x)
#     x = UpSampling2D(size=(2, 2))(x)
#     x = Concatenate()([x, skip_tensor])
#     return x


In [ ]:
# from tensorflow.keras.models import Model

# def build_unet_with_cbam(input_shape, num_classes):
#     inputs = Input(input_shape)

#     # Encoder
#     e1 = encoder_block(inputs, 64)
#     p1 = MaxPooling2D(pool_size=(2, 2))(e1)

#     e2 = encoder_block(p1, 128)
#     p2 = MaxPooling2D(pool_size=(2, 2))(e2)

#     e3 = encoder_block(p2, 256)
#     p3 = MaxPooling2D(pool_size=(2, 2))(e3)

#     e4 = encoder_block(p3, 512)
#     p4 = MaxPooling2D(pool_size=(2, 2))(e4)

#     # Bottleneck
#     b = encoder_block(p4, 1024)

#     # Decoder
#     d4 = decoder_block(b, e4, 512)
#     d3 = decoder_block(d4, e3, 256)
#     d2 = decoder_block(d3, e2, 128)
#     d1 = decoder_block(d2, e1, 64)

#     outputs = Conv2D(num_classes, (1, 1), activation='sigmoid')(d1)

#     model = Model(inputs, outputs)
#     return model


In [ ]:
# # Set input shape and number of classes
# input_shape = (256, 256, 3)  # Adjust this according to your input shape
# num_classes = 1  # Adjust this according to your number of classes

# # Build the model
# model = build_unet_with_cbam(input_shape, num_classes)

# # Compile the model
# model.compile(optimizer='adam', loss=bce_dice_loss, metrics=[iou, F1, 'accuracy'])

# # Print the model summary
# model.summary()

# # Load training and validation data
# train_path = '/content/drive/My Drive/sayandeep/Processed_Dataset/training'  # Update this with the actual path
# validation_path = '/content/drive/My Drive/sayandeep/Processed_Dataset/validation'  # Update this with the actual path
# batch_size = 16  # Adjust batch size as needed
# image_size = 256  # Adjust image size as needed

# train_ids = os.listdir(os.path.join(train_path, "images", "img"))
# val_ids = os.listdir(os.path.join(validation_path, "images", "img"))

# train_gen = DataGen(train_ids, train_path, batch_size=batch_size, image_size=image_size)
# val_gen = DataGen(val_ids, validation_path, batch_size=batch_size, image_size=image_size)

# # Callbacks
# checkpoint = ModelCheckpoint('/content/drive/My Drive/sayandeep/sayandeep_unet_cbam.h5', save_best_only=True, monitor='val_loss', mode='min')
# lr_scheduler = LearningRateScheduler(scheduler)
# reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=5, min_lr=0.00001)

# # Train the model
# history = model.fit(
#     train_gen,
#     validation_data=val_gen,
#     epochs=50,
#     callbacks=[checkpoint, lr_scheduler, reduce_lr]
# )

# # Save the model as TensorFlow SavedModel
# model_save_path_tf = '/content/drive/My Drive/sayandeep/sayandeep_cbam_unet'
# model.save(model_save_path_tf, save_format='tf')
# print(f"Model saved to {model_save_path_tf}")


In [ ]:
# import tensorflow as tf
# import os
# import numpy as np
# import matplotlib.pyplot as plt
# from sklearn.metrics import confusion_matrix, jaccard_score
# from tensorflow.keras.preprocessing.image import ImageDataGenerator

# # Function to load the saved model
# def load_model(model_save_path_tf):
#     model = tf.keras.models.load_model(model_save_path_tf, custom_objects={'bce_dice_loss': bce_dice_loss, 'iou': iou, 'F1': F1})
#     print(f"Model loaded from {model_save_path_tf}")
#     return model

# # Function to evaluate the model
# def evaluate_model(model, test_gen):
#     results = {}
#     y_true_all = []
#     y_pred_all = []
#     num_tests = len(test_gen)
#     for i in range(num_tests):
#         x, y_true = test_gen[i]
#         y_pred_prob = model.predict(x)
#         y_pred = (y_pred_prob > 0.8).astype(int)
#         y_true = (y_true > 0.5).astype(int)
#         y_true_all.extend(y_true.flatten())
#         y_pred_all.extend(y_pred.flatten())
#     y_true_all = np.array(y_true_all)
#     y_pred_all = np.array(y_pred_all)
#     tn, fp, fn, tp = confusion_matrix(y_true_all, y_pred_all).ravel()
#     precision = tp / (tp + fp)
#     recall = tp / (tp + fn)
#     specificity = tn / (tn + fp)
#     iou = jaccard_score(y_true_all, y_pred_all)
#     out = model.evaluate(test_gen, steps=num_tests)
#     last_metric = out[-1]
#     results = {
#         'Precision': precision,
#         'Recall': recall,
#         'Specificity': specificity,
#         'IoU': iou,
#         'Eval Metric': last_metric
#     }
#     print(results)
#     return results

# # Function to visualize model performance
# def plot_metrics(history):
#     # Plot training & validation loss values
#     plt.figure(figsize=(12, 4))
#     plt.subplot(1, 2, 1)
#     plt.plot(history.history['loss'])
#     plt.plot(history.history['val_loss'])
#     plt.title('Model loss')
#     plt.ylabel('Loss')
#     plt.xlabel('Epoch')
#     plt.legend(['Train', 'Validation'], loc='upper left')

#     # Plot training & validation accuracy values
#     plt.subplot(1, 2, 2)
#     plt.plot(history.history['accuracy'])
#     plt.plot(history.history['val_accuracy'])
#     plt.title('Model accuracy')
#     plt.ylabel('Accuracy')
#     plt.xlabel('Epoch')
#     plt.legend(['Train', 'Validation'], loc='upper left')

#     plt.show()

# # Function to visualize predictions
# def visualize_predictions(model, test_gen, num_images=5):
#     for i in range(num_images):
#         x, y_true = test_gen[i]
#         y_pred_prob = model.predict(x)
#         y_pred = (y_pred_prob > 0.8).astype(int)

#         plt.figure(figsize=(15, 5))

#         # Original Image
#         plt.subplot(1, 3, 1)
#         plt.imshow(np.reshape(x[0], (image_size, image_size, 3)))
#         plt.title('Original Image')

#         # Ground Truth
#         plt.subplot(1, 3, 2)
#         plt.imshow(np.reshape(y_true[0], (image_size, image_size)), cmap='gray')
#         plt.title('Ground Truth')

#         # Predicted Mask
#         plt.subplot(1, 3, 3)
#         plt.imshow(np.reshape(y_pred[0], (image_size, image_size)), cmap='gray')
#         plt.title('Predicted Mask')

#         plt.show()

# # Load the saved model
# model_save_path_tf = '/content/drive/My Drive/sayandeep/sayandeep_cbam_unet'
# model = load_model(model_save_path_tf)

# # Load test data
# test_path = '/content/drive/My Drive/sayandeep/Processed_Dataset/testing'  # Update this with the actual path
# batch_size = 16  # Adjust batch size as needed
# image_size = 256  # Adjust image size as needed

# test_ids = os.listdir(os.path.join(test_path, "images", "img"))
# test_gen = DataGen(test_ids, test_path, batch_size=batch_size, image_size=image_size)

# # Evaluate the model
# results = evaluate_model(model, test_gen)

# # Visualize predictions
# visualize_predictions(model, test_gen, num_images=5)


KeyboardInterrupt: 

In [ ]:
import tensorflow as tf
from tensorflow.keras import layers, models
import os
import numpy as np
import matplotlib.pyplot as plt
from sklearn.metrics import confusion_matrix, jaccard_score
from tensorflow.keras.preprocessing.image import load_img, img_to_array
from tensorflow.keras.utils import Sequence
from tensorflow.keras.callbacks import ModelCheckpoint, LearningRateScheduler, ReduceLROnPlateau


In [ ]:
def cbam_block(input_tensor, ratio=8):
    channel = input_tensor.shape[-1]
    shared_layer_one = layers.Dense(channel // ratio, activation='relu', kernel_initializer='he_normal', use_bias=True, bias_initializer='zeros')
    shared_layer_two = layers.Dense(channel, kernel_initializer='he_normal', use_bias=True, bias_initializer='zeros')

    avg_pool = layers.GlobalAveragePooling2D()(input_tensor)
    avg_pool = layers.Reshape((1, 1, channel))(avg_pool)
    avg_pool = shared_layer_one(avg_pool)
    avg_pool = shared_layer_two(avg_pool)

    max_pool = layers.GlobalMaxPooling2D()(input_tensor)
    max_pool = layers.Reshape((1, 1, channel))(max_pool)
    max_pool = shared_layer_one(max_pool)
    max_pool = shared_layer_two(max_pool)

    cbam_feature = layers.Add()([avg_pool, max_pool])
    cbam_feature = layers.Activation('sigmoid')(cbam_feature)

    cbam_feature = layers.Multiply()([input_tensor, cbam_feature])

    # Spatial attention
    avg_pool = layers.Lambda(lambda x: tf.reduce_mean(x, axis=-1, keepdims=True))(cbam_feature)
    max_pool = layers.Lambda(lambda x: tf.reduce_max(x, axis=-1, keepdims=True))(cbam_feature)
    concat = layers.Concatenate(axis=-1)([avg_pool, max_pool])
    cbam_feature = layers.Conv2D(filters=1, kernel_size=7, strides=1, padding='same', activation='sigmoid', kernel_initializer='he_normal', use_bias=False)(concat)

    cbam_feature = layers.Multiply()([cbam_feature, cbam_feature])

    return cbam_feature


In [ ]:
def se_block(input_tensor, ratio=8):
    channel = input_tensor.shape[-1]
    se_feature = layers.GlobalAveragePooling2D()(input_tensor)
    se_feature = layers.Reshape((1, 1, channel))(se_feature)
    se_feature = layers.Dense(channel // ratio, activation='relu', kernel_initializer='he_normal', use_bias=True, bias_initializer='zeros')(se_feature)
    se_feature = layers.Dense(channel, activation='sigmoid', kernel_initializer='he_normal', use_bias=True, bias_initializer='zeros')(se_feature)
    se_feature = layers.Multiply()([input_tensor, se_feature])
    return se_feature


In [ ]:
def scse_block(input_tensor, ratio=8):
    se_feature = se_block(input_tensor, ratio)

    avg_pool = layers.Lambda(lambda x: tf.reduce_mean(x, axis=-1, keepdims=True))(input_tensor)
    max_pool = layers.Lambda(lambda x: tf.reduce_max(x, axis=-1, keepdims=True))(input_tensor)
    concat = layers.Concatenate(axis=-1)([avg_pool, max_pool])
    spatial_se = layers.Conv2D(filters=1, kernel_size=1, strides=1, padding='same', activation='sigmoid', kernel_initializer='he_normal', use_bias=False)(concat)

    scse_feature = layers.Add()([layers.Multiply()([input_tensor, spatial_se]), se_feature])

    return scse_feature


In [ ]:
def unet_with_attention(input_shape):
    inputs = layers.Input(input_shape)

    # Encoder
    c1 = layers.Conv2D(64, (3, 3), activation='relu', padding='same')(inputs)
    c1 = layers.Conv2D(64, (3, 3), activation='relu', padding='same')(c1)
    c1 = scse_block(c1)
    p1 = layers.MaxPooling2D((2, 2))(c1)

    c2 = layers.Conv2D(128, (3, 3), activation='relu', padding='same')(p1)
    c2 = layers.Conv2D(128, (3, 3), activation='relu', padding='same')(c2)
    c2 = cbam_block(c2)
    p2 = layers.MaxPooling2D((2, 2))(c2)

    c3 = layers.Conv2D(256, (3, 3), activation='relu', padding='same')(p2)
    c3 = layers.Conv2D(256, (3, 3), activation='relu', padding='same')(c3)
    c3 = scse_block(c3)
    p3 = layers.MaxPooling2D((2, 2))(c3)

    c4 = layers.Conv2D(512, (3, 3), activation='relu', padding='same')(p3)
    c4 = layers.Conv2D(512, (3, 3), activation='relu', padding='same')(c4)
    c4 = cbam_block(c4)
    p4 = layers.MaxPooling2D((2, 2))(c4)

    # Bottleneck
    c5 = layers.Conv2D(1024, (3, 3), activation='relu', padding='same')(p4)
    c5 = layers.Conv2D(1024, (3, 3), activation='relu', padding='same')(c5)
    c5 = scse_block(c5)

    # Decoder
    u6 = layers.Conv2DTranspose(512, (2, 2), strides=(2, 2), padding='same')(c5)
    u6 = layers.concatenate([u6, c4])
    c6 = layers.Conv2D(512, (3, 3), activation='relu', padding='same')(u6)
    c6 = layers.Conv2D(512, (3, 3), activation='relu', padding='same')(c6)
    c6 = cbam_block(c6)

    u7 = layers.Conv2DTranspose(256, (2, 2), strides=(2, 2), padding='same')(c6)
    u7 = layers.concatenate([u7, c3])
    c7 = layers.Conv2D(256, (3, 3), activation='relu', padding='same')(u7)
    c7 = layers.Conv2D(256, (3, 3), activation='relu', padding='same')(c7)
    c7 = scse_block(c7)

    u8 = layers.Conv2DTranspose(128, (2, 2), strides=(2, 2), padding='same')(c7)
    u8 = layers.concatenate([u8, c2])
    c8 = layers.Conv2D(128, (3, 3), activation='relu', padding='same')(u8)
    c8 = layers.Conv2D(128, (3, 3), activation='relu', padding='same')(c8)
    c8 = cbam_block(c8)

    u9 = layers.Conv2DTranspose(64, (2, 2), strides=(2, 2), padding='same')(c8)
    u9 = layers.concatenate([u9, c1])
    c9 = layers.Conv2D(64, (3, 3), activation='relu', padding='same')(u9)
    c9 = layers.Conv2D(64, (3, 3), activation='relu', padding='same')(c9)
    c9 = scse_block(c9)

    outputs = layers.Conv2D(1, (1, 1), activation='sigmoid')(c9)

    model = models.Model(inputs=[inputs], outputs=[outputs])

    return model


In [ ]:
class DataGen(Sequence):
    def __init__(self, ids, path, batch_size=2, image_size=256):
        self.ids = ids
        self.path = path
        self.batch_size = batch_size
        self.image_size = image_size
        self.on_epoch_end()

    def __load__(self, id_name):
        image_path = os.path.join(self.path, "images", "img", id_name)
        mask_path = os.path.join(self.path, "mask", "img", id_name)
        image = cv2.imread(image_path)
        if image is None or image.size == 0:
            return None, None
        image = cv2.resize(image, (self.image_size, self.image_size)) / 255.0

        mask = cv2.imread(mask_path, cv2.IMREAD_GRAYSCALE)
        if mask is None or mask.size == 0:
            return None, None
        mask = cv2.resize(mask, (self.image_size, self.image_size)) / 255.0
        mask = np.expand_dims(mask, axis=-1)

        return image, mask

    def __getitem__(self, index):
        files_batch = self.ids[index*self.batch_size : (index+1)*self.batch_size]
        images, masks = [], []
        for id_name in files_batch:
            img, msk = self.__load__(id_name)
            if img is not None and msk is not None:
                images.append(img)
                masks.append(msk)
        return np.array(images), np.array(masks)

    def on_epoch_end(self):
        np.random.shuffle(self.ids)

    def __len__(self):
        return int(np.ceil(len(self.ids) / float(self.batch_size)))

In [ ]:
def iou(y_true, y_pred, smooth=1):
    intersection = K.sum(K.abs(y_true * y_pred), axis=[1,2,3])
    union = K.sum(y_true,[1,2,3])+K.sum(y_pred,[1,2,3])-intersection
    iou = K.mean((intersection + smooth) / (union + smooth), axis=0)
    return iou

def F1(y_true, y_pred, smooth=1):
    intersection = K.sum(y_true * y_pred, axis=[1, 2, 3])
    union = K.sum(y_true, axis=[1, 2, 3]) + K.sum(y_pred, axis=[1, 2, 3])
    dice = K.mean((2. * intersection + smooth) / (union + smooth), axis=0)
    return dice

def dice_coef(y_true, y_pred, smooth = 0.00001):
    y_true_f = K.flatten(y_true)
    y_pred_f = K.flatten(y_pred)
    intersection = K.sum(y_true_f * y_pred_f)
    return (2. * intersection + smooth) / (K.sum(y_true_f) + K.sum(y_pred_f) + smooth)

def recall(y_true, y_pred):
    true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
    possible_positives = K.sum(K.round(K.clip(y_true, 0, 1)))
    recall = true_positives / (possible_positives + K.epsilon())
    return recall

def precision(y_true, y_pred):
    true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
    predicted_positives = K.sum(K.round(K.clip(y_pred, 0, 1)))
    precision = true_positives / (predicted_positives + K.epsilon())
    return precision

def dice_coef_loss(y_true, y_pred):
    return -dice_coef(y_true, y_pred)

def dice_loss(y_true, y_pred):
    y_true_f = tf.keras.backend.flatten(y_true)
    y_pred_f = tf.keras.backend.flatten(y_pred)
    intersection = tf.keras.backend.sum(y_true_f * y_pred_f)
    return 1 - (2. * intersection + 1) / (tf.keras.backend.sum(y_true_f) + tf.keras.backend.sum(y_pred_f) + 1)

def bce_dice_loss(y_true, y_pred):
    bce = tf.keras.losses.BinaryCrossentropy()(y_true, y_pred)
    dice = dice_loss(y_true, y_pred)
    return bce + dice

#till here it is mostly redundant, below is real code but like i said, keep.
def load_test_data(test_path, image_size, batch_size=10):
    test_ids = os.listdir(os.path.join(test_path, "images", "img"))
    gen = DataGen(test_ids, test_path, batch_size=batch_size, image_size=image_size)
    print("Test Data Loaded")
    return gen, len(test_ids)

def scheduler(epoch, lr):
    if epoch < 30:
        lr = 0.001
        return lr
    if epoch < 50:
        return 0.0005
    return 0.0001

def visualize_output(x, y, y_pred, tot, idx, fig, title_flag=False):
    contours1, hierarchy = cv2.findContours(y[0].astype(np.uint8), cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_NONE)
    contours2, hierarchy = cv2.findContours(y_pred[0].astype(np.uint8), cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_NONE)
    cv2.drawContours(x[0], contours1, -1, (0, 255, 0), 3)
    cv2.drawContours(x[0], contours2, -1, (255, 0, 0), 3)
    ax = fig.add_subplot(tot, 5, idx)
    ax.imshow(np.reshape(x[0], (image_size, image_size, 3)))

def evaluate_model(gen, model):
    results = {}
    y_true_all = []
    y_pred_all = []
    num_tests = len(gen)
    for i in range(num_tests):
        x, y_true = gen[i]
        y_pred_prob = model.predict(x)
        y_pred = (y_pred_prob > 0.8).astype(int)
        y_true = (y_true > 0.5).astype(int)
        y_true_all.extend(y_true.flatten())
        y_pred_all.extend(y_pred.flatten())
    y_true_all = np.array(y_true_all)
    y_pred_all = np.array(y_pred_all)
    tn, fp, fn, tp = confusion_matrix(y_true_all, y_pred_all).ravel()
    precision = tp / (tp + fp)
    recall = tp / (tp + fn)
    specificity = tn / (tn + fp)
    iou = jaccard_score(y_true_all, y_pred_all)
    out = model.evaluate(gen, steps=num_tests)
    last_metric = out[-1]
    results = {
        'Precision': precision,
        'Recall': recall,
        'Specificity': specificity,
        'IoU': iou,
        'Eval Metric': last_metric
    }
    print(results)

In [ ]:
# class DataGen(Sequence):
#     def __init__(self, image_ids, path, batch_size, image_size):
#         self.image_ids = image_ids
#         self.path = path
#         self.batch_size = batch_size
#         self.image_size = image_size
#         self.on_epoch_end()

#     def __len__(self):
#         return int(np.floor(len(self.image_ids) / self.batch_size))

#     def __getitem__(self, index):
#         batch_ids = self.image_ids[index * self.batch_size:(index + 1) * self.batch_size]
#         images = np.zeros((self.batch_size, self.image_size, self.image_size, 3), dtype=np.float32)
#         masks = np.zeros((self.batch_size, self.image_size, self.image_size, 1), dtype=np.float32)

#         for i, id in enumerate(batch_ids):
#             # Updated path construction for images
#             image_path = os.path.join(self.path, 'images',  id)
#             # Updated path construction for masks
#             mask_path = os.path.join(self.path, 'masks', id)

#             # Debugging: Print paths
#             print(f"Loading image: {image_path}")
#             print(f"Loading mask: {mask_path}")

#             image = img_to_array(load_img(image_path, target_size=(self.image_size, self.image_size)))
#             mask = img_to_array(load_img(mask_path, color_mode="grayscale", target_size=(self.image_size, self.image_size)))
#             images[i] = image / 255.0
#             masks[i] = mask / 255.0

#         return images, masks

#     def on_epoch_end(self):
#         np.random.shuffle(self.image_ids)

In [ ]:
# def bce_dice_loss(y_true, y_pred):
#     bce = tf.keras.losses.BinaryCrossentropy()(y_true, y_pred)
#     y_true_f = tf.keras.backend.flatten(y_true)
#     y_pred_f = tf.keras.backend.flatten(y_pred)
#     intersection = tf.keras.backend.sum(y_true_f * y_pred_f)
#     dice_loss = 1 - (2. * intersection + 1) / (tf.keras.backend.sum(y_true_f) + tf.keras.backend.sum(y_pred_f) + 1)
#     return bce + dice_loss

# def iou(y_true, y_pred, smooth=1):
#     intersection = tf.keras.backend.sum(tf.keras.backend.abs(y_true * y_pred), axis=[1, 2, 3])
#     union = tf.keras.backend.sum(y_true, axis=[1, 2, 3]) + tf.keras.backend.sum(y_pred, axis=[1, 2, 3]) - intersection
#     return tf.keras.backend.mean((intersection + smooth) / (union + smooth), axis=0)

# def F1(y_true, y_pred, smooth=1):
#     intersection = tf.keras.backend.sum(y_true * y_pred, axis=[1, 2, 3])
#     union = tf.keras.backend.sum(y_true, axis=[1, 2, 3]) + tf.keras.backend.sum(y_pred, axis=[1, 2, 3])
#     return tf.keras.backend.mean((2. * intersection + smooth) / (union + smooth), axis=0)


In [ ]:
train_path = '/content/drive/MyDrive/sayandeep/Processed_Dataset/training/'  # Update this with the actual path
validation_path = '/content/drive/MyDrive/sayandeep/Processed_Dataset/validation/'
# Training parameters
batch_size = 16  # Adjust batch size as needed
image_size = 256  # Adjust image size as needed
epochs = 50  # Adjust number of epochs as needed

# Load training and validation data
train_ids = os.listdir(os.path.join(train_path, "images", "img"))
val_ids = os.listdir(os.path.join(validation_path, "images", "img"))

train_gen = DataGen(train_ids, train_path, batch_size=batch_size, image_size=image_size)
val_gen = DataGen(val_ids, validation_path, batch_size=batch_size, image_size=image_size)

# Define the model
input_shape = (image_size, image_size, 3)
model = unet_with_attention(input_shape)

# Compile the model
model.compile(optimizer='adam', loss=bce_dice_loss, metrics=['accuracy', iou, F1])

# Callbacks
checkpoint = ModelCheckpoint('/content/drive/My Drive/sayandeep/sayandeep_unet_cbam_se_best_model.h5', save_best_only=True, monitor='val_loss', mode='min')
lr_scheduler = LearningRateScheduler(scheduler)
reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=5, min_lr=0.00001)

# Train the model
history = model.fit(
    train_gen,
    validation_data=val_gen,
    epochs=epochs,
    callbacks=[checkpoint, lr_scheduler, reduce_lr]
)

model_save_path_tf = '/content/drive/My Drive/sayandeep/sayandeep_cbam_unet_se'
model.save(model_save_path_tf, save_format='tf')
print(f"Model saved to {model_save_path_tf}")


Epoch 1/50
307/307 [==============================] - ETA: 0s - loss: 0.9725 - accuracy: 0.9693 - iou: 0.0686 - F1: 0.1191 

/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


307/307 [==============================] - 4443s 14s/step - loss: 0.9725 - accuracy: 0.9693 - iou: 0.0686 - F1: 0.1191 - val_loss: 0.9249 - val_accuracy: 0.9779 - val_iou: 0.0734 - val_F1: 0.1269 - lr: 0.0010
Epoch 2/50
307/307 [==============================] - 65s 212ms/step - loss: 0.9152 - accuracy: 0.9679 - iou: 0.0920 - F1: 0.1532 - val_loss: 0.9412 - val_accuracy: 0.9765 - val_iou: 0.0636 - val_F1: 0.1085 - lr: 0.0010
Epoch 3/50
307/307 [==============================] - 67s 219ms/step - loss: 0.9125 - accuracy: 0.9672 - iou: 0.0936 - F1: 0.1559 - val_loss: 0.8891 - val_accuracy: 0.9724 - val_iou: 0.0996 - val_F1: 0.1620 - lr: 0.0010
Epoch 4/50
307/307 [==============================] - 71s 229ms/step - loss: 0.8985 - accuracy: 0.9698 - iou: 0.0985 - F1: 0.1625 - val_loss: 0.8792 - val_accuracy: 0.9732 - val_iou: 0.1004 - val_F1: 0.1657 - lr: 0.0010
Epoch 5/50
307/307 [==============================] - 65s 208ms/step - loss: 0.8936 - accuracy: 0.9710 - iou: 0.1007 - F1: 0.1657 -

In [ ]:
import os
import numpy as np
import tensorflow as tf
from tensorflow.keras.models import load_model
from sklearn.metrics import confusion_matrix, jaccard_score, precision_score, recall_score

# Custom loss and metrics functions
def bce_dice_loss(y_true, y_pred):
    bce = tf.keras.losses.BinaryCrossentropy()(y_true, y_pred)
    y_true_f = tf.keras.backend.flatten(y_true)
    y_pred_f = tf.keras.backend.flatten(y_pred)
    intersection = tf.keras.backend.sum(y_true_f * y_pred_f)
    dice_loss = 1 - (2. * intersection + 1) / (tf.keras.backend.sum(y_true_f) + tf.keras.backend.sum(y_pred_f) + 1)
    return bce + dice_loss

def iou(y_true, y_pred, smooth=1):
    intersection = tf.keras.backend.sum(tf.keras.backend.abs(y_true * y_pred), axis=[1, 2, 3])
    union = tf.keras.backend.sum(y_true, axis=[1, 2, 3]) + tf.keras.backend.sum(y_pred, axis=[1, 2, 3]) - intersection
    return tf.keras.backend.mean((intersection + smooth) / (union + smooth), axis=0)

def F1(y_true, y_pred, smooth=1):
    intersection = tf.keras.backend.sum(y_true * y_pred, axis=[1, 2, 3])
    union = tf.keras.backend.sum(y_true, axis=[1, 2, 3]) + tf.keras.backend.sum(y_pred, axis=[1, 2, 3])
    return tf.keras.backend.mean((2. * intersection + smooth) / (union + smooth), axis=0)

# Function to load the saved model
def load_model_custom(model_save_path_tf):
    return tf.keras.models.load_model(model_save_path_tf, custom_objects={'bce_dice_loss': bce_dice_loss, 'iou': iou, 'F1': F1})

# Evaluate the model
def evaluate_model(model, gen):
    results = {}
    y_true_all = []
    y_pred_all = []
    num_tests = len(gen)
    for i in range(num_tests):
        x, y_true = gen[i]
        y_pred_prob = model.predict(x)
        y_pred = (y_pred_prob > 0.5).astype(int)
        y_true = (y_true > 0.5).astype(int)
        y_true_all.extend(y_true.flatten())
        y_pred_all.extend(y_pred.flatten())
    y_true_all = np.array(y_true_all)
    y_pred_all = np.array(y_pred_all)
    tn, fp, fn, tp = confusion_matrix(y_true_all, y_pred_all).ravel()
    precision = precision_score(y_true_all, y_pred_all)
    recall = recall_score(y_true_all, y_pred_all)
    specificity = tn / (tn + fp)
    iou_score = jaccard_score(y_true_all, y_pred_all)
    out = model.evaluate(gen, steps=num_tests)
    last_metric = out[-1]
    results = {
        'Precision': precision,
        'Recall': recall,
        'Specificity': specificity,
        'IoU': iou_score,
        'Eval Metric': last_metric
    }
    return results

# Example usage
model_save_path_tf = '/content/drive/My Drive/sayandeep/sayandeep_cbam_unet'  # Update this with your model path
test_path = '/content/drive/My Drive/sayandeep/Processed_Dataset/testing'  # Update this with the actual path
image_size = 256  # Adjust image size as needed

# Load the model
model = load_model_custom(model_save_path_tf)

# Load test data
test_gen, num_test_samples = load_test_data(test_path, image_size)

# Evaluate the model
results = evaluate_model(model, test_gen)
print("Evaluation Results:", results)


Test Data Loaded
27/27 [==============================] - 5s 136ms/step - loss: 0.3591 - iou: 0.5255 - F1: 0.6605 - accuracy: 0.9914
Evaluation Results: {'Precision': 0.7394526087496752, 'Recall': 0.7098953532505557, 'Specificity': 0.9959665260952612, 'IoU': 0.5678559346292971, 'Eval Metric': 0.9914266467094421}


In [ ]:
import os
import numpy as np
import tensorflow as tf
from tensorflow.keras.models import load_model
from sklearn.metrics import confusion_matrix, jaccard_score, precision_score, recall_score

# Custom loss and metrics functions
def bce_dice_loss(y_true, y_pred):
    bce = tf.keras.losses.BinaryCrossentropy()(y_true, y_pred)
    y_true_f = tf.keras.backend.flatten(y_true)
    y_pred_f = tf.keras.backend.flatten(y_pred)
    intersection = tf.keras.backend.sum(y_true_f * y_pred_f)
    dice_loss = 1 - (2. * intersection + 1) / (tf.keras.backend.sum(y_true_f) + tf.keras.backend.sum(y_pred_f) + 1)
    return bce + dice_loss

def iou(y_true, y_pred, smooth=1):
    intersection = tf.keras.backend.sum(tf.keras.backend.abs(y_true * y_pred), axis=[1, 2, 3])
    union = tf.keras.backend.sum(y_true, axis=[1, 2, 3]) + tf.keras.backend.sum(y_pred, axis=[1, 2, 3]) - intersection
    return tf.keras.backend.mean((intersection + smooth) / (union + smooth), axis=0)

def F1(y_true, y_pred, smooth=1):
    intersection = tf.keras.backend.sum(y_true * y_pred, axis=[1, 2, 3])
    union = tf.keras.backend.sum(y_true, axis=[1, 2, 3]) + tf.keras.backend.sum(y_pred, axis=[1, 2, 3])
    return tf.keras.backend.mean((2. * intersection + smooth) / (union + smooth), axis=0)

# Function to load the saved model
def load_model_custom(model_save_path_tf):
    return tf.keras.models.load_model(model_save_path_tf, custom_objects={'bce_dice_loss': bce_dice_loss, 'iou': iou, 'F1': F1})

# Evaluate the model
def evaluate_model(model, gen):
    results = {}
    y_true_all = []
    y_pred_all = []
    num_tests = len(gen)
    for i in range(num_tests):
        x, y_true = gen[i]
        y_pred_prob = model.predict(x)
        y_pred = (y_pred_prob > 0.5).astype(int)
        y_true = (y_true > 0.5).astype(int)
        y_true_all.extend(y_true.flatten())
        y_pred_all.extend(y_pred.flatten())
    y_true_all = np.array(y_true_all)
    y_pred_all = np.array(y_pred_all)
    tn, fp, fn, tp = confusion_matrix(y_true_all, y_pred_all).ravel()
    precision = precision_score(y_true_all, y_pred_all)
    recall = recall_score(y_true_all, y_pred_all)
    specificity = tn / (tn + fp)
    iou_score = jaccard_score(y_true_all, y_pred_all)
    out = model.evaluate(gen, steps=num_tests)
    last_metric = out[-1]
    results = {
        'Precision': precision,
        'Recall': recall,
        'Specificity': specificity,
        'IoU': iou_score,
        'Eval Metric': last_metric
    }
    return results

# Example usage
model_save_path_tf = '/content/drive/My Drive/sayandeep/sayandeep_unet'  # Update this with your model path
test_path = '/content/drive/My Drive/sayandeep/Processed_Dataset/testing'  # Update this with the actual path
image_size = 256  # Adjust image size as needed

# Load the model
model = load_model_custom(model_save_path_tf)

# Load test data
test_gen, num_test_samples = load_test_data(test_path, image_size)

# Evaluate the model
results = evaluate_model(model, test_gen)
print("Evaluation Results:", results)

Test Data Loaded
27/27 [==============================] - 5s 164ms/step - loss: 0.0591 - accuracy: 0.9912
Evaluation Results: {'Precision': 0.7324245551561065, 'Recall': 0.7003524372390609, 'Specificity': 0.9958741965096503, 'IoU': 0.55766822958492, 'Eval Metric': 0.9911845326423645}


In [ ]:
import os
import numpy as np
import tensorflow as tf
from tensorflow.keras.models import load_model
from sklearn.metrics import confusion_matrix, jaccard_score, precision_score, recall_score

# Custom loss and metrics functions
def bce_dice_loss(y_true, y_pred):
    bce = tf.keras.losses.BinaryCrossentropy()(y_true, y_pred)
    y_true_f = tf.keras.backend.flatten(y_true)
    y_pred_f = tf.keras.backend.flatten(y_pred)
    intersection = tf.keras.backend.sum(y_true_f * y_pred_f)
    dice_loss = 1 - (2. * intersection + 1) / (tf.keras.backend.sum(y_true_f) + tf.keras.backend.sum(y_pred_f) + 1)
    return bce + dice_loss

def iou(y_true, y_pred, smooth=1):
    intersection = tf.keras.backend.sum(tf.keras.backend.abs(y_true * y_pred), axis=[1, 2, 3])
    union = tf.keras.backend.sum(y_true, axis=[1, 2, 3]) + tf.keras.backend.sum(y_pred, axis=[1, 2, 3]) - intersection
    return tf.keras.backend.mean((intersection + smooth) / (union + smooth), axis=0)

def F1(y_true, y_pred, smooth=1):
    intersection = tf.keras.backend.sum(y_true * y_pred, axis=[1, 2, 3])
    union = tf.keras.backend.sum(y_true, axis=[1, 2, 3]) + tf.keras.backend.sum(y_pred, axis=[1, 2, 3])
    return tf.keras.backend.mean((2. * intersection + smooth) / (union + smooth), axis=0)

# Function to load the saved model
def load_model_custom(model_save_path_tf):
    return tf.keras.models.load_model(model_save_path_tf, custom_objects={'bce_dice_loss': bce_dice_loss, 'iou': iou, 'F1': F1})

# Evaluate the model
def evaluate_model(model, gen):
    results = {}
    y_true_all = []
    y_pred_all = []
    num_tests = len(gen)
    for i in range(num_tests):
        x, y_true = gen[i]
        y_pred_prob = model.predict(x)
        y_pred = (y_pred_prob > 0.5).astype(int)
        y_true = (y_true > 0.5).astype(int)
        y_true_all.extend(y_true.flatten())
        y_pred_all.extend(y_pred.flatten())
    y_true_all = np.array(y_true_all)
    y_pred_all = np.array(y_pred_all)
    tn, fp, fn, tp = confusion_matrix(y_true_all, y_pred_all).ravel()
    precision = precision_score(y_true_all, y_pred_all)
    recall = recall_score(y_true_all, y_pred_all)
    specificity = tn / (tn + fp)
    iou_score = jaccard_score(y_true_all, y_pred_all)
    out = model.evaluate(gen, steps=num_tests)
    last_metric = out[-1]
    results = {
        'Precision': precision,
        'Recall': recall,
        'Specificity': specificity,
        'IoU': iou_score,
        'Eval Metric': last_metric
    }
    return results

# Example usage
model_save_path_tf = '/content/drive/My Drive/sayandeep/sayandeep_unet_attention'  # Update this with your model path
test_path = '/content/drive/My Drive/sayandeep/Processed_Dataset/testing'  # Update this with the actual path
image_size = 256  # Adjust image size as needed

# Load the model
model = load_model_custom(model_save_path_tf)

# Load test data
test_gen, num_test_samples = load_test_data(test_path, image_size)

# Evaluate the model
results = evaluate_model(model, test_gen)
print("Evaluation Results:", results)

Test Data Loaded
27/27 [==============================] - 4s 128ms/step - loss: 0.3616 - iou: 0.5223 - F1: 0.6546 - accuracy: 0.9919
Evaluation Results: {'Precision': 0.7962251842831981, 'Recall': 0.6618156843608234, 'Specificity': 0.9972687579589822, 'IoU': 0.5659563155714648, 'Eval Metric': 0.9919454455375671}


In [ ]:
import os
import numpy as np
import tensorflow as tf
from tensorflow.keras.models import load_model
from sklearn.metrics import confusion_matrix, jaccard_score, precision_score, recall_score

# Custom loss and metrics functions
def bce_dice_loss(y_true, y_pred):
    bce = tf.keras.losses.BinaryCrossentropy()(y_true, y_pred)
    y_true_f = tf.keras.backend.flatten(y_true)
    y_pred_f = tf.keras.backend.flatten(y_pred)
    intersection = tf.keras.backend.sum(y_true_f * y_pred_f)
    dice_loss = 1 - (2. * intersection + 1) / (tf.keras.backend.sum(y_true_f) + tf.keras.backend.sum(y_pred_f) + 1)
    return bce + dice_loss

def iou(y_true, y_pred, smooth=1):
    intersection = tf.keras.backend.sum(tf.keras.backend.abs(y_true * y_pred), axis=[1, 2, 3])
    union = tf.keras.backend.sum(y_true, axis=[1, 2, 3]) + tf.keras.backend.sum(y_pred, axis=[1, 2, 3]) - intersection
    return tf.keras.backend.mean((intersection + smooth) / (union + smooth), axis=0)

def F1(y_true, y_pred, smooth=1):
    intersection = tf.keras.backend.sum(y_true * y_pred, axis=[1, 2, 3])
    union = tf.keras.backend.sum(y_true, axis=[1, 2, 3]) + tf.keras.backend.sum(y_pred, axis=[1, 2, 3])
    return tf.keras.backend.mean((2. * intersection + smooth) / (union + smooth), axis=0)

# Function to load the saved model
def load_model_custom(model_save_path_tf):
    return tf.keras.models.load_model(model_save_path_tf, custom_objects={'bce_dice_loss': bce_dice_loss, 'iou': iou, 'F1': F1})

# Evaluate the model
def evaluate_model(model, gen):
    results = {}
    y_true_all = []
    y_pred_all = []
    num_tests = len(gen)
    for i in range(num_tests):
        x, y_true = gen[i]
        y_pred_prob = model.predict(x)
        y_pred = (y_pred_prob > 0.5).astype(int)
        y_true = (y_true > 0.5).astype(int)
        y_true_all.extend(y_true.flatten())
        y_pred_all.extend(y_pred.flatten())
    y_true_all = np.array(y_true_all)
    y_pred_all = np.array(y_pred_all)
    tn, fp, fn, tp = confusion_matrix(y_true_all, y_pred_all).ravel()
    precision = precision_score(y_true_all, y_pred_all)
    recall = recall_score(y_true_all, y_pred_all)
    specificity = tn / (tn + fp)
    iou_score = jaccard_score(y_true_all, y_pred_all)
    out = model.evaluate(gen, steps=num_tests)
    last_metric = out[-1]
    results = {
        'Precision': precision,
        'Recall': recall,
        'Specificity': specificity,
        'IoU': iou_score,
        'Eval Metric': last_metric
    }
    return results

# Example usage
model_save_path_tf = '/content/drive/My Drive/sayandeep/sayandeep_cbam_unet_se'  # Update this with your model path
test_path = '/content/drive/My Drive/sayandeep/Processed_Dataset/testing'  # Update this with the actual path
image_size = 256  # Adjust image size as needed

# Load the model
model = load_model_custom(model_save_path_tf)

# Load test data
test_gen, num_test_samples = load_test_data(test_path, image_size)

# Evaluate the model
results = evaluate_model(model, test_gen)
print("Evaluation Results:", results)

Test Data Loaded
27/27 [==============================] - 6s 166ms/step - loss: 0.7286 - accuracy: 0.9825 - iou: 0.1508 - F1: 0.2375
Evaluation Results: {'Precision': 0.43040817981086094, 'Recall': 0.3262339821793273, 'Specificity': 0.993038209351623, 'IoU': 0.22785995793789654, 'Eval Metric': 0.23753945529460907}


In [7]:
import os
import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt
from tensorflow.keras.models import load_model
from sklearn.metrics import confusion_matrix, jaccard_score, precision_score, recall_score
from tensorflow.keras.preprocessing.image import load_img, img_to_array

# Custom loss and metrics functions
def bce_dice_loss(y_true, y_pred):
    bce = tf.keras.losses.BinaryCrossentropy()(y_true, y_pred)
    y_true_f = tf.keras.backend.flatten(y_true)
    y_pred_f = tf.keras.backend.flatten(y_pred)
    intersection = tf.keras.backend.sum(y_true_f * y_pred_f)
    dice_loss = 1 - (2. * intersection + 1) / (tf.keras.backend.sum(y_true_f) + tf.keras.backend.sum(y_pred_f) + 1)
    return bce + dice_loss

def iou(y_true, y_pred, smooth=1):
    intersection = tf.keras.backend.sum(tf.keras.backend.abs(y_true * y_pred), axis=[1, 2, 3])
    union = tf.keras.backend.sum(y_true, axis=[1, 2, 3]) + tf.keras.backend.sum(y_pred, axis=[1, 2, 3]) - intersection
    return tf.keras.backend.mean((intersection + smooth) / (union + smooth), axis=0)

def F1(y_true, y_pred, smooth=1):
    intersection = tf.keras.backend.sum(y_true * y_pred, axis=[1, 2, 3])
    union = tf.keras.backend.sum(y_true, axis=[1, 2, 3]) + tf.keras.backend.sum(y_pred, axis=[1, 2, 3])
    return tf.keras.backend.mean((2. * intersection + smooth) / (union + smooth), axis=0)

# Function to load the saved model
def load_model_custom(model_save_path_tf):
    return tf.keras.models.load_model(model_save_path_tf, custom_objects={'bce_dice_loss': bce_dice_loss, 'iou': iou, 'F1': F1})



# Function to load test data
def load_test_data(test_path, image_size, batch_size=10):
    test_ids = os.listdir(os.path.join(test_path, "images", "img"))
    gen = DataGen(test_ids, test_path, batch_size=batch_size, image_size=image_size)
    print("Test Data Loaded")
    return gen, len(test_ids)

# Evaluate the model
def evaluate_model(model, gen):
    results = {}
    y_true_all = []
    y_pred_all = []
    num_tests = len(gen)
    for i in range(num_tests):
        x, y_true = gen[i]
        y_pred_prob = model.predict(x)
        y_pred = (y_pred_prob > 0.5).astype(int)
        y_true = (y_true > 0.5).astype(int)
        y_true_all.extend(y_true.flatten())
        y_pred_all.extend(y_pred.flatten())
    y_true_all = np.array(y_true_all)
    y_pred_all = np.array(y_pred_all)
    tn, fp, fn, tp = confusion_matrix(y_true_all, y_pred_all).ravel()
    precision = precision_score(y_true_all, y_pred_all)
    recall = recall_score(y_true_all, y_pred_all)
    specificity = tn / (tn + fp)
    iou_score = jaccard_score(y_true_all, y_pred_all)
    out = model.evaluate(gen, steps=num_tests)
    last_metric = out[-1]
    results = {
        'Precision': precision,
        'Recall': recall,
        'Specificity': specificity,
        'IoU': iou_score,
        'Eval Metric': last_metric
    }
    return results

# Visualize the model predictions
def visualize_predictions(model, gen, num_images=5):
    fig, axes = plt.subplots(num_images, 3, figsize=(15, num_images * 5))
    for i in range(num_images):
        x, y_true = gen[i]
        y_pred_prob = model.predict(x)
        y_pred = (y_pred_prob > 0.5).astype(int)

        axes[i, 0].imshow(x[0])
        axes[i, 0].set_title("Input Image")
        axes[i, 0].axis("off")

        axes[i, 1].imshow(y_true[0].squeeze(), cmap="gray")
        axes[i, 1].set_title("True Mask")
        axes[i, 1].axis("off")

        axes[i, 2].imshow(y_pred[0].squeeze(), cmap="gray")
        axes[i, 2].set_title("Predicted Mask")
        axes[i, 2].axis("off")

    plt.tight_layout()
    plt.show()

# Example usage
model_save_path_tf = '/content/drive/My Drive/sayandeep/sayandeep_cbam_unet'  # Update this with your model path
test_path = '/content/drive/My Drive/sayandeep/Processed_Dataset/testing'  # Update this with the actual path
image_size = 256  # Adjust image size as needed

# Load the model
model = load_model_custom(model_save_path_tf)

# Load test data
test_gen, num_test_samples = load_test_data(test_path, image_size)

# Evaluate the model
results = evaluate_model(model, test_gen)
print("Evaluation Results:", results)

# Visualize the predictions
visualize_predictions(model, test_gen, num_images=5)


Test Data Loaded


FileNotFoundError: [Errno 2] No such file or directory: '/content/drive/My Drive/sayandeep/Processed_Dataset/testing/masks/mask/200.png'

In [ ]:
#Most important Code in the Entire Code. ALWAYS DO RUN ALL
from google.colab import runtime
runtime.unassign()